In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("creditcard.csv")

In [ ]:
df.shape

(284807, 31)

In [ ]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
encoding_dim = 32

In [ ]:
X_train.shape[1]

30

In [ ]:
encoder = models.Sequential([
    layers.Dense(encoding_dim, activation='relu', input_shape=(X_train.shape[1],)),
])

decoder = models.Sequential([
    layers.Dense(X_train.shape[1], activation='sigmoid'),
])

In [ ]:
autoencoder = models.Sequential([
    encoder,
    decoder,
])

In [ ]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
autoencoder.fit(X_train, X_train, epochs=10, batch_size=64, shuffle=True, validation_data=(X_test, X_test))

Epoch 1/10
2671/2671 [==============================] - 10s 4ms/step - loss: 0.6913 - accuracy: 0.8779 - val_loss: 0.2008 - val_accuracy: 0.0892
Epoch 2/10
2671/2671 [==============================] - 8s 3ms/step - loss: 0.6735 - accuracy: 0.9315 - val_loss: 0.2019 - val_accuracy: 0.0869
Epoch 3/10
2671/2671 [==============================] - 10s 4ms/step - loss: 0.6684 - accuracy: 0.9332 - val_loss: 0.2049 - val_accuracy: 0.0886
Epoch 4/10
2671/2671 [==============================] - 9s 4ms/step - loss: 0.6684 - accuracy: 0.9330 - val_loss: 0.2055 - val_accuracy: 0.0906
Epoch 5/10
2671/2671 [==============================] - 8s 3ms/step - loss: 0.6683 - accuracy: 0.9325 - val_loss: 0.2050 - val_accuracy: 0.0880
Epoch 6/10
2671/2671 [==============================] - 13s 5ms/step - loss: 0.6682 - accuracy: 0.9336 - val_loss: 0.2049 - val_accuracy: 0.0892
Epoch 7/10
2671/2671 [==============================] - 11s 4ms/step - loss: 0.6682 - accuracy: 0.9334 - val_loss: 0.2050 - val_accur

In [ ]:
loss = autoencoder.evaluate(X_test, X_test)
print(f'Test Loss: {loss}')
print(loss[0])
print(loss[1])

3561/3561 [==============================] - 9s 2ms/step - loss: 0.6486 - accuracy: 0.9280
Test Loss: [0.6485553979873657, 0.9279864430427551]
0.6485553979873657
0.9279864430427551


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Evaluate the model on the test set
decoded_data = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - decoded_data, 2), axis=1)
threshold = np.percentile(mse, 95)  # Adjust the percentile as needed for tuning sensitivity

# Identify outliers based on the threshold
outliers = mse > threshold

# Print evaluation metrics for anomaly detection
print("Confusion Matrix:\n", confusion_matrix(y_test, outliers))
print("\nClassification Report:\n", classification_report(y_test, outliers))

3561/3561 [==============================] - 6s 2ms/step
Confusion Matrix:
 [[108204   5528]
 [    22    169]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.95      0.97    113732
           1       0.03      0.88      0.06       191

    accuracy                           0.95    113923
   macro avg       0.51      0.92      0.52    113923
weighted avg       1.00      0.95      0.97    113923



In [ ]:
# Print the number of outliers and anomalies
num_outliers = np.sum(outliers)
num_anomalies = np.sum(y_test[outliers] == 1)  # Assuming 1 represents the positive class

print(f'Number of outliers: {num_outliers}')
print(f'Number of anomalies: {num_anomalies}')

Number of outliers: 5697
Number of anomalies: 169
